In [6]:
import cpickle
from pathlib import Path
import torch

cp_path = Path("/mnt/data/users/keller/Uni/trf_training_tut/scripts/pytorch-lightning/lightning_runs/rocstories/default/0/checkpoints/epoch=2-step=6867.ckpt")

ModuleNotFoundError: No module named 'cpickle'

In [7]:
with cp_path.open("rb") as f:
    obj = torch.load(f)

In [12]:
obj.keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])

In [14]:
obj["hyper_parameters"]

{'seed': 42,
 'train_batch_size': 16,
 'val_batch_size': 32,
 'logger': True,
 'checkpoint_callback': True,
 'default_root_dir': 'lightning_runs/rocstories',
 'gradient_clip_val': 1.0,
 'gradient_clip_algorithm': 'norm',
 'process_position': 0,
 'num_nodes': 1,
 'num_processes': 10,
 'devices': None,
 'gpus': 1,
 'auto_select_gpus': False,
 'tpu_cores': None,
 'ipus': None,
 'log_gpu_memory': None,
 'progress_bar_refresh_rate': None,
 'overfit_batches': 0.0,
 'track_grad_norm': -1,
 'check_val_every_n_epoch': 1,
 'fast_dev_run': False,
 'accumulate_grad_batches': 1,
 'max_epochs': 3,
 'min_epochs': None,
 'max_steps': None,
 'min_steps': None,
 'max_time': None,
 'limit_train_batches': 1.0,
 'limit_val_batches': 1.0,
 'limit_test_batches': 1.0,
 'limit_predict_batches': 1.0,
 'val_check_interval': 200,
 'flush_logs_every_n_steps': 100,
 'log_every_n_steps': 50,
 'accelerator': 'ddp_spawn',
 'sync_batchnorm': False,
 'precision': 32,
 'weights_summary': 'top',
 'weights_save_path': None